# Juice.jl: a library for advanced probabilistic inference

What is Juice.jl: a Julia library enabling tractable advanced probabilistic inference by leveraging logical and probabilistic circuits!

What I am going to talk about:
    - Probabilistic circuits
       - Loading an already learned circuit
    - "Usual" tractable probabilistic inference routines
       - Complete Evidence (EVI)
       - Marginals (MAR)
       - Conditionals (CON)
    - Some advanced probabilistic reasoning: 
       - Expected Prediction

In [1]:
using CSV
using Statistics

In [2]:
# Loading Juice Libraries, in this demo we mainly use ProbabilisticCircuits
using LogicCircuits
using ProbabilisticCircuits

In [3]:
# You can skip this part. Includes helper functions to make partial observations from arrays of strings
# so its easier to present.

# Make one ovservation from list of string describing the observation
#
# For example, ["smoker", "male"] sets 
#   1) The mentioned features to the correct values.
#   2) Every feature not mentioned to missing values. 
FEATURES = 36;
function make_one_observation(obs)
    result = Int8.(ones(FEATURES) * -1)
    for k in obs
        # Smoking
        if lowercase(k) == "smoker"
            result[7:8] .= [0, 1]
        elseif lowercase(k) == "!smoker"
            result[7:8] .= [1, 0]
        # Gender
        elseif lowercase(k) == "male"
            result[13:14] .= [1, 0]
        elseif lowercase(k) == "female"
            result[13:14] .= [0, 1]
        # Region
        elseif lowercase(k) == "southeast"
            result[9:12] .= [0, 0, 1, 0]
        elseif lowercase(k) == "southwest"
            result[9:12] .= [0, 1, 0, 0]
        # Child
        elseif lowercase(k) == "1-child"
            result[1:6] .= [0,1,0,0,0,0]
        end
    end
    result
end;

function make_obs(obs)
    count = size(obs)[1]
    result = Int8.(ones(count, 36) * -1)
    for i=1:count
        result[i, :] .= make_one_observation(obs[i])
    end
    XData(result)
end;

# Exploratory probabilistic analysis 

### Insurance dataset
yearly health insurance costs of individuals living in the USA

In [4]:
## load Insurance Dataset
train_x = Bool.(Matrix(CSV.read("insurance/insurance_train_x.csv")));
train_y = Matrix(CSV.read("insurance/insurance_train_y.csv"));

test_x = Bool.(Matrix(CSV.read("insurance/insurance_test_x.csv")));
test_y = Matrix(CSV.read("insurance/insurance_test_y.csv"));

valid_x = Bool.(Matrix(CSV.read("insurance/insurance_valid_x.csv")));
valid_y = Matrix(CSV.read("insurance/insurance_valid_y.csv"));

Here for the purpose of this demo, we load a pretrained probabilistic circuit:

In [5]:
pc = load_prob_circuit(zoo_psdd_file("insurance.psdd"));
println("Probablistic Circuit with $(size(pc)[1]) nodes")

Probablistic Circuit with 27493 nodes


### Structural Properties

By enforcing some properties on the structure of the circuits we can enable tractable operations.


Properties:
- Structured decomposable
- Smooth
- Deterministic

Check this tutorial on Probabilistic Circuits: http://starai.cs.ucla.edu/slides/Stanford19.pdf

### EVI: Complete Evidence Query

$ P(x) $

In [6]:
lls_train = log_proba(pc, XData(train_x));

In [7]:
println("size: $(length(lls_train))\nmean: $(mean(lls_train)) \n min: $(minimum(lls_train)) \n max: $(maximum(lls_train))")

size: 935
mean: -9.711713749299689 
 min: -14.475790561571328 
 max: -6.676430127955419


### MAR: Marginal Query (partial evidence)

$ P(X^o) $

$$ P(X^o) = \sum_{x^m} P(X^o X^m) $$

In probabilistic circuits, given some properties, we can do marginal queries with any subset of variables in time linear to size of the circuit instead of enumerating all possible worlds.

In [8]:
marg_data = make_obs( [["smoker"], 
                       ["female"], 
                       ["female", "smoker"], 
                       ["southeast", "male", "1-child", "smoker"]],
                    )
lls = log_proba(pc, marg_data);

#### Q: Probablity of being smoker?

In [9]:
marg_data = make_obs([["smoker"]])
lls = log_proba(pc, marg_data);
exp.(lls[1])

0.18403566812744454

#### Q: Probability of being female smoker?

In [10]:
marg_data = make_obs([["female", "smoker"]])
lls = log_proba(pc, marg_data);
exp.(lls[1])

0.09623782748058206

####  Q: Probability of being male smoker with one child living in the southeast?

In [11]:
marg_data = make_obs([["southeast", "male", "1-child", "smoker"]])
lls = log_proba(pc, marg_data);
exp.(lls[1])

0.00096392685052339

# Exploratory predictive analysis

What about reasoning about predictive models such as regression models:


We are interested in computing **expected predictions**

- Appears all the time in machine learning, such as handling missing data
- We can do this tractably!


$$ \Large \mathbb{E}_{\mathbf{x}^m \sim p(\mathbf{x}^m\ \mid\ \mathbf{x}^o )}\left[f( \mathbf{x}^o \mathbf{x}^m)\right] $$

- In above equation $ \mathbf{x}^m $ = missing features, and $ \mathbf{x}^o $ = observed features.


- Expected Prediction useful for:
  - Handling missing values at test time
  - Reasoning aobut behaviour of predictive models

Check out our paper: On Tractable Computation of Expected Predictions (NeurIPS 2019, https://arxiv.org/abs/1910.02182)

In [12]:
# Load Regression Circuit
rc = load_logistic_circuit(zoo_lc_file("insurance.circuit"), 1);

In [13]:
println("Regression Circuit with $(size(rc)[1]) nodes")

Regression Circuit with 1148 nodes


## Sample Queries

### Q1: How different are the insurance costs between smokers and non smokers?

In [14]:
data = make_obs([["!smoker"], 
                 ["smoker"]])
exps, exp_cache = Expectation(pc, rc, data);

In [15]:
println("Smoker    : \$ $(exps[2])");
println("Non-Smoker: \$ $(exps[1])");
println("Difference: \$ $(exps[2] - exps[1])");

Smoker    : $ 31355.32630488977
Non-Smoker: $ 8741.74725831065
Difference: $ 22613.57904657912


### Q2: Is the predictive model biased by gender?

In [16]:
data = make_obs([["male"],
                 ["female"]])
exps, exp_cache = Expectation(pc, rc, data);

In [17]:
println("Female  : \$ $(exps[2])");
println("Male    : \$ $(exps[1])");
println("Diff    : \$ $(exps[2] - exps[1])");

Female  : $ 14170.12546933539
Male    : $ 13196.548926381856
Diff    : $ 973.5765429535331


### Q3:  Expecation and standard devation of few subpopulations

In [18]:
data = make_obs( [["southeast", "male", "1-child", "smoker"], 
                 ["southwest", "male", "1-child", "smoker"]])
exps, exp_cache = Expectation(pc, rc, data);

In [19]:
# Computes the second moment
mom2, mom_cache = Moment(pc, rc, data, 2);

### Computing Standard Deviation

In [20]:
stds = sqrt.( mom2 - exps.^2 );

#### Living in South East, Smoker, Male, One child

In [21]:
println("mu: $(round(exps[1])), std = $(round(stds[1]))")

mu: 30975.0, std = 11229.0


#### Living in South West, Smoker, Male, One child

In [22]:
println("mu: $(round(exps[2])), std = $(round(stds[2]))")

mu: 27251.0, std = 7717.0
